In [2]:
%logstop
%logstart -ortq ~/.logs/DS_Data_Munging.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

# Data Munging

## Relational Data

The simplest type of data we have see might consist a single table with a some columns and some rows. This sort of data is easy to analyze and compute and we generally want to reduce our data to a single table before we start running machine learning algorithms. Yet, real world data doesn't necessarily fit into this paradigm.  Most real world data is messy and complicated which doesn't fit well into a tabular format and we will have to do some work to reduce this complexity.  Additionally, in many case we can reduce our memory cost by not keeping data in a single table, but instead in a set of data structures with defined relations between them. 

Here we will explore a bit of data and see how combining different sets of data can help us generate useful features.

First we need some data.  We will make use of some data from Wikipedia and we will use the pandas `read_html` function to scrape the data from a particular webpage.  We will study the top 10 companies in the Fortune Global 500 which conveniently have [their own Wikipedia page](https://en.wikipedia.org/w/index.php?title=Fortune_Global_500&oldid=855890446).

We will download the data in tabular form, but work with it as a list of dictionaries, this will allow us to get used to working with unstructured data.

In [3]:
import pandas as pd
import json
df = pd.read_html('https://en.wikipedia.org/w/index.php?title=Fortune_Global_500&oldid=855890446', header=0)[0]
fortune_500 = json.loads(df.to_json(orient="records"))
df

,Rank,Company,Country,Industry,Revenue in USD
0,1,Walmart,United States,Retail,$500 billion
1,2,State Grid,China,Power,$349 billion
2,3,Sinopec Group,China,Petroleum,$327 billion
3,4,China National Petroleum,China,Petroleum,$326 billion
4,5,Royal Dutch Shell,Netherlands,Petroleum,$312 billion
5,6,Toyota Motor,Japan,Automobiles,$265 billion
6,7,Volkswagen,Germany,Automobiles,$260 billion
7,8,BP,United Kingdom,Petroleum,$245 billion
8,9,Exxon Mobil,United States,Petroleum,$244 billion
9,10,Berkshire Hathaway,United States,Products,$242 billion


Lets look at the data.

In [4]:
fortune_500

[{'Rank': 1,
  'Company': 'Walmart',
  'Country': 'United States',
  'Industry': 'Retail',
  'Revenue in USD': '$500 billion'},
 {'Rank': 2,
  'Company': 'State Grid',
  'Country': 'China',
  'Industry': 'Power',
  'Revenue in USD': '$349 billion'},
 {'Rank': 3,
  'Company': 'Sinopec Group',
  'Country': 'China',
  'Industry': 'Petroleum',
  'Revenue in USD': '$327 billion'},
 {'Rank': 4,
  'Company': 'China National Petroleum',
  'Country': 'China',
  'Industry': 'Petroleum',
  'Revenue in USD': '$326 billion'},
 {'Rank': 5,
  'Company': 'Royal Dutch Shell',
  'Country': 'Netherlands',
  'Industry': 'Petroleum',
  'Revenue in USD': '$312 billion'},
 {'Rank': 6,
  'Company': 'Toyota Motor',
  'Country': 'Japan',
  'Industry': 'Automobiles',
  'Revenue in USD': '$265 billion'},
 {'Rank': 7,
  'Company': 'Volkswagen',
  'Country': 'Germany',
  'Industry': 'Automobiles',
  'Revenue in USD': '$260 billion'},
 {'Rank': 8,
  'Company': 'BP',
  'Country': 'United Kingdom',
  'Industry': 'Petr

This is a great start to our analysis, however, there really isn't that much information here, we will need to bring in additional data sources to get any further understanding of these companies.

The first question we might want to ask is how many employees does it take to get that revenue, in other words, what is the revenue per employee?  Luckily, we can use Wikipedia to get that data as well, we have scraped this data manually (all from Wikipedia) and created the following dictionary.

In [5]:
other_data = [
    {"name": "Walmart",
     "employees": 2300000,
     "year founded": 1962
    },
    {"name": "State Grid Corporation of China",
     "employees": 927839,
     "year founded": 2002},
    {"name": "China Petrochemical Corporation",
     "employees":358571,
     "year founded": 1998
     },
    {"name": "China National Petroleum Corporation",
     "employees": 1636532,
     "year founded": 1988},
    {"name": "Toyota Motor Corporation",
     "employees": 364445,
     "year founded": 1937},
    {"name": "Volkswagen AG",
     "employees": 642292,
     "year founded": 1937},
    {"name": "Royal Dutch Shell",
     "employees": 92000,
     "year founded": 1907},
    {"name": "Berkshire Hathaway Inc.",
     "employees":377000,
     "year founded": 1839},
    {"name": "Apple Inc.",
     "employees": 123000,
     "year founded": 1976},
    {"name": "Exxon Mobile Corporation",
     "employees": 69600,
     "year founded": 1999},
    {"name": "BP plc",
     "employees": 74000,
     "year founded": 1908}
]

Some data have a slightly different name than in our original set, so we will keep a dictionary of mappings between the two.  Notice, we only include the mapping in the dictionary if there is a difference.

In [6]:
mapping = {
    'Apple': 'Apple Inc.',
    'BP': 'BP plc',
    'Berkshire Hathaway': 'Berkshire Hathaway Inc.',
    'China National Petroleum': 'China National Petroleum Corporation',
    'Exxon Mobil': 'Exxon Mobile Corporation',
    'Sinopec Group': 'China Petrochemical Corporation',
    'State Grid': 'State Grid Corporation of China',
    'Toyota Motor': 'Toyota Motor Corporation',
    'Volkswagen': 'Volkswagen AG'
}

This data is one to one, meaning the data contained in one source only aligns with a single element in the other source, thus we should be able to put these together.  However, we know that the data isn't in a great form to be joined at the moment.  This is for two reasons

1. All the names will not align (we need to use our mapping)
2. The `list` structure is not optimized for looking through elements. 

While for 10 elements the second reason won't really matter, for larger data sets such performance considerations are extremely important.  We can turn this list of dictionaries into a dictionary of dictionaries, so we can quickly access each element of the data.

In [7]:
dict_data = {k["name"] : k for k in other_data}
dict_data

{'Walmart': {'name': 'Walmart', 'employees': 2300000, 'year founded': 1962},
 'State Grid Corporation of China': {'name': 'State Grid Corporation of China',
  'employees': 927839,
  'year founded': 2002},
 'China Petrochemical Corporation': {'name': 'China Petrochemical Corporation',
  'employees': 358571,
  'year founded': 1998},
 'China National Petroleum Corporation': {'name': 'China National Petroleum Corporation',
  'employees': 1636532,
  'year founded': 1988},
 'Toyota Motor Corporation': {'name': 'Toyota Motor Corporation',
  'employees': 364445,
  'year founded': 1937},
 'Volkswagen AG': {'name': 'Volkswagen AG',
  'employees': 642292,
  'year founded': 1937},
 'Royal Dutch Shell': {'name': 'Royal Dutch Shell',
  'employees': 92000,
  'year founded': 1907},
 'Berkshire Hathaway Inc.': {'name': 'Berkshire Hathaway Inc.',
  'employees': 377000,
  'year founded': 1839},
 'Apple Inc.': {'name': 'Apple Inc.',
  'employees': 123000,
  'year founded': 1976},
 'Exxon Mobile Corporatio

**Question:** If we had many entries in `other_data`, we could display a small piece by printing `other_data[:5]`. With dataframes we might use `df.head()`. Can you think of a way to print out a small piece of a dictionary?

Now we can easily compute the revenue per employee, we need to map the "Company" value in our original data with the "name" column of this other data, but we also need to use the mapping to ensure the columns will line up.  We in general don't want to mutate our original data, so lets make a new list of dictionaries with this new feature (revenue per employee).  On the course of doing this, we will need to handle converting some numbers like `$500 Billion` to a numeric value.  Lets create a function to do this.

In [10]:
def convert_revenue(x):
    return float(x.lstrip('$').rstrip('billion')) * 1e9

assert convert_revenue('$500 billion') == 500e9

Now we should be able to create a few functions to compute this revenue per employee and create a data list.

In [11]:
def rev_per_emp(company):
    name = company[u'Company']
    n_employees = dict_data[mapping.get(name, name)].get('employees')
    company['rev per emp'] = convert_revenue(company[u'Revenue in USD'])/n_employees
    return company

def compute_copy(d, func):
    return func({k:v for k,v in d.items()})

data = list(map(lambda x : compute_copy(x, rev_per_emp), fortune_500))

Lets take a look at our new data and also the old data to ensure we didn't mutate anything.

In [12]:
data[:2]

[{'Rank': 1,
  'Company': 'Walmart',
  'Country': 'United States',
  'Industry': 'Retail',
  'Revenue in USD': '$500 billion',
  'rev per emp': 217391.30434782608},
 {'Rank': 2,
  'Company': 'State Grid',
  'Country': 'China',
  'Industry': 'Power',
  'Revenue in USD': '$349 billion',
  'rev per emp': 376142.84374767606}]

In [13]:
fortune_500[:2]

[{'Rank': 1,
  'Company': 'Walmart',
  'Country': 'United States',
  'Industry': 'Retail',
  'Revenue in USD': '$500 billion'},
 {'Rank': 2,
  'Company': 'State Grid',
  'Country': 'China',
  'Industry': 'Power',
  'Revenue in USD': '$349 billion'}]

Now we can sort these values.  We first can select out on the elements we care about and then sort that list.

In [14]:
rev_per_emp = sorted([(i[u'Company'], i['rev per emp']) for i in data], 
                   key=lambda x : x[1],
                   reverse=True)
rev_per_emp

[('Exxon Mobil', 3505747.1264367816),
 ('Royal Dutch Shell', 3391304.347826087),
 ('BP', 3310810.810810811),
 ('Sinopec Group', 911953.2812190612),
 ('Toyota Motor', 727133.0379069544),
 ('Berkshire Hathaway', 641909.8143236075),
 ('Volkswagen', 404800.30889377417),
 ('State Grid', 376142.84374767606),
 ('Walmart', 217391.30434782608),
 ('China National Petroleum', 199201.72657790987)]

This results in a much different order.  What does this tell us about the companies?

Now lets pull in some other data (this is data science, more data is always better!).  We can see that these companies are in a few different industries, let find out which ones.

In [15]:
from collections import Counter
Counter(i[u'Industry'] for i in data)

Counter({'Retail': 1,
         'Power': 1,
         'Petroleum': 5,
         'Automobiles': 2,
         'Products': 1})

One thing we might want to know is what sort of market share they have of the specific industry to which they belong.  Let's look at the two industries that categorize the 6 of the top 10, `Automobiles` and `Petroleum`.  We can select only those elements of our data to work with.

In [16]:
sub_data = [i for i in data if i[u'Industry'] in [u'Automobiles', u'Petroleum']]
sub_data

[{'Rank': 3,
  'Company': 'Sinopec Group',
  'Country': 'China',
  'Industry': 'Petroleum',
  'Revenue in USD': '$327 billion',
  'rev per emp': 911953.2812190612},
 {'Rank': 4,
  'Company': 'China National Petroleum',
  'Country': 'China',
  'Industry': 'Petroleum',
  'Revenue in USD': '$326 billion',
  'rev per emp': 199201.72657790987},
 {'Rank': 5,
  'Company': 'Royal Dutch Shell',
  'Country': 'Netherlands',
  'Industry': 'Petroleum',
  'Revenue in USD': '$312 billion',
  'rev per emp': 3391304.347826087},
 {'Rank': 6,
  'Company': 'Toyota Motor',
  'Country': 'Japan',
  'Industry': 'Automobiles',
  'Revenue in USD': '$265 billion',
  'rev per emp': 727133.0379069544},
 {'Rank': 7,
  'Company': 'Volkswagen',
  'Country': 'Germany',
  'Industry': 'Automobiles',
  'Revenue in USD': '$260 billion',
  'rev per emp': 404800.30889377417},
 {'Rank': 8,
  'Company': 'BP',
  'Country': 'United Kingdom',
  'Industry': 'Petroleum',
  'Revenue in USD': '$245 billion',
  'rev per emp': 3310810

It might be the case that the each particular category has a different relevant metric for market share.  For example, we could look at total revenue for a car company or we could look at cars produced.  

So for the automobile industry we will look at the percent total of cars produced.  We can get this data again from Wikipedia.

In [17]:
df_list = pd.read_html("https://en.wikipedia.org/w/index.php?title=Automotive_industry&oldid=875776152", header=0)
car_totals = json.loads(df_list[0].to_json(orient="records"))
car_by_man = json.loads(df_list[2].to_json(orient='records'))

In [18]:
car_totals[:2]

[{'Year': 1997, 'Production': 54434000, 'Change': '—', 'Source': '[17]'},
 {'Year': 1998, 'Production': 52987000, 'Change': '2.7%', 'Source': '[17]'}]

In [19]:
car_by_man[:2]

[{'Rank': 1, 'Group': 'Toyota', 'Country': 'Japan', 'Vehicles': 10213486},
 {'Rank': 2,
  'Group': 'Volkswagen Group',
  'Country': 'Germany',
  'Vehicles': 10126281}]

Now lets get only the groups we care about and divide by the total production which we will take as the latest year.

In [20]:
total_prod = sorted((i[u"Year"], i[u'Production']) for i in car_totals)[-1][1]
total_prod

97302534

Now we can find the market share for each of the car companies. We will keep track of a market share dictionary.  We will again need to keep track of some slight name differences.

In [21]:
car_by_man_dict = {i[u'Group']:i[u'Vehicles'] for i in car_by_man}
market_share = {}
for name, orig_name in zip(['Toyota', 'Volkswagen Group'], ['Toyota', 'Volkswagen']):
    market_share[orig_name] = car_by_man_dict[name]/ float(total_prod)
    
market_share

{'Toyota': 0.1049662899837737, 'Volkswagen': 0.10407006460900597}

Now we can do the same for the Petroleum industry, but in this case, lets compute the market share by revenue.  On Wikipedia, we can find a list of oil companies by revenue.  Although its not a complete list, it has enough companies that we don't expect the companies left off the list to contribute greatly to our analysis.

In [22]:
rev = pd.read_html("https://en.wikipedia.org/w/index.php?title=List_of_largest_oil_and_gas_companies_by_revenue&oldid=871711850", header=1)[0]
rev = rev.iloc[:, 1:3]
rev.columns = ['Company', 'Revenue']
rev = rev[~(rev['Company'] == 'Company name')]
oil_data = json.loads(rev.to_json(orient="records"))
oil_data[:2]

[{'Company': 'Saudi Aramco', 'Revenue': '465.49'},
 {'Company': 'Sinopec Group', 'Revenue': '448.00'}]

Now we can compute the totals and market share.  Since the data here might be slightly different (perhaps older) than our original data, we will compute the market share of each company within this data set, then pull out the numbers we care about.

In [23]:
total = sum([float(i[u'Revenue'].rstrip('*')) for i in oil_data])
shares = {i[u'Company']:float(i[u'Revenue'].rstrip('*'))/total for i in oil_data}
print(total)

5482.090000000001


Now we can pull out the companies we care about in the petroleum industry.

In [24]:
petro_companies = [i[u'Company'] for i in data if i['Industry'] == u'Petroleum']
petro_companies

['Sinopec Group',
 'China National Petroleum',
 'Royal Dutch Shell',
 'BP',
 'Exxon Mobil']

Lets check if these are all in the our shares dictionary.

In [26]:
[(i, i in shares) for i in petro_companies]

[('Sinopec Group', True),
 ('China National Petroleum', False),
 ('Royal Dutch Shell', True),
 ('BP', True),
 ('Exxon Mobil', True)]

Some of these companies are directly there, and looking through our dictionary, we can see the others are there without exact names.

In [27]:
shares.keys()

dict_keys(['Saudi Aramco', 'Sinopec Group', 'China National Petroleum Corporation', 'Exxon Mobil', 'Royal Dutch Shell', 'Kuwait Petroleum Corporation', 'BP', 'Total SA', 'Lukoil', 'Eni', 'Valero Energy', 'Chevron Corporation', 'PDVSA', 'Pemex', 'National Iranian Oil Company', 'Gazprom', 'Phillips 66', 'Petronas', 'China National Offshore Oil', 'Marathon Petroleum', 'PTT', 'Rosneft', 'JX Holdings', 'Engie', 'Petrobras', 'Equinor', 'Indian Oil Corporation', 'Sonatrach', 'Reliance Industries', 'Pertamina', 'SOCAR', 'GS Caltex', 'Enterprise Products', 'Repsol', 'Centrica', 'Bharat Petroleum', 'OMV Group', 'Idemitsu Kosan', 'Hindustan Petroleum', 'PKN Orlen', 'Schlumberger', 'Suncor Energy', 'Hellenic Petroleum', 'ConocoPhillips', 'Motor Oil Hellas', 'Ecopetrol', 'CEPSA', 'MOL'])

So lets make a fuzzy match, this will be a pretty simple one where it will try to match words in a name and take the maximum number of matches.

In [28]:
def fuzzy_match(word, s):
    words = set(word.split(' '))
    overlaps = [(k, len(v.intersection(words))) for k, v in s.items()]
    return max(overlaps, key=lambda x : x[1])[0]

In [29]:
split_names = {i: set(i.split(' ')) for i in shares.keys()}
for i in petro_companies:
    match = fuzzy_match(i, split_names)
    print("matched {} to {}".format(i, match))
    market_share[i] = shares[match]

matched Sinopec Group to Sinopec Group
matched China National Petroleum to China National Petroleum Corporation
matched Royal Dutch Shell to Royal Dutch Shell
matched BP to BP
matched Exxon Mobil to Exxon Mobil


In [30]:
market_share

{'Toyota': 0.1049662899837737,
 'Volkswagen': 0.10407006460900597,
 'Sinopec Group': 0.0817206576323993,
 'China National Petroleum': 0.07818550954106918,
 'Royal Dutch Shell': 0.04833922828702191,
 'BP': 0.040641434197541446,
 'Exxon Mobil': 0.0490506357976611}

## By industry
We have some nice examples of data munging, now lets see an example of keeping data in a relational fashion.  Lets say we want to add another feature which is the growth of each industry.  If we were to store this data as a single quantity, we would be saving a bunch of extra information, we would be much better off extracting this information and keeping it in a single table so we are not replicating by industry.

## With Pandas

Now we can also perform these same computations with Pandas, lets see how this compares.

In [ ]:
df = pd.read_html('https://en.wikipedia.org/w/index.php?title=Fortune_Global_500&oldid=855890446', header=0)[0]
df

In [ ]:
df['rev'] = df['Revenue in USD'].apply(convert_revenue)
df['employees'] = df['Company'].apply(lambda x : dict_data[mapping.get(x, x)].get('employees'))
df['rev_per_employee'] = df['rev'] / df['employees'].astype(float)
df.sort_values(by='rev_per_employee', ascending=False)

In [ ]:
df_list = pd.read_html("https://en.wikipedia.org/w/index.php?title=Automotive_industry&oldid=875776152", header=0)
df_totals = df_list[0]
df_by_man = df_list[2]

In [ ]:
total_prod = df_totals.sort_values(by='Year').iloc[-1]['Production']
total_prod

In [ ]:
df_by_man['share'] = df_by_man['Vehicles'].astype(float) / total_prod
market_share = df_by_man.set_index('Group')['share'][['Toyota', 'Volkswagen Group']]
market_share

In [ ]:
rev = pd.read_html("https://en.wikipedia.org/w/index.php?title=List_of_largest_oil_and_gas_companies_by_revenue&oldid=871711850", header=1)[0]
rev = rev.iloc[:, 1:3]
rev.columns = ['Company', 'Revenue']
rev = rev[~(rev['Company'] == 'Company name')]
rev

In [ ]:
rev['rev_clean'] = rev['Revenue'].apply(lambda x : float(x.rstrip('*')))
total = rev['rev_clean'].sum()
total

In [ ]:
rev['share'] = rev['rev_clean'] / total
rev

In [ ]:
rev = rev[rev['Company'].isin(['Exxon Mobil', 'Sinopec', 'China National Petroleum Corporation', 'Royal Dutch Shell'])].copy()
rev

In [ ]:
# do fuzzy search
split_names = {i: set(i.split(' ')) for i in df['Company']}

def fuzzy(word):
    return fuzzy_match(word, split_names)

rev['name'] = rev['Company'].apply(fuzzy)
rev

In [ ]:
ms2 = df.merge(rev[['share', 'name']], left_on='Company', right_on='name')

Now we want to put these together and get only the company and the market share.

In [ ]:
ms = market_share.reset_index()[['Group','share']]
ms.columns = ['Company', 'share']
pd.concat([ms, ms2[['Company', 'share']]])

*Copyright &copy; 2019 The Data Incubator.  All rights reserved.*